In [1]:
import pandas as pd
import re
import json
df = pd.read_csv("public/data/All_patient_data_fre/element_patient_counts.csv")
df["Element #"] = df["Element #"].astype(int)
results = df.set_index("Element #")["Number of Patients"].to_dict()
print(results)
df

{41: 6, 42: 6, 44: 2, 45: 2, 47: 4, 48: 11, 49: 6, 50: 2, 51: 1, 54: 12, 55: 11, 57: 6, 59: 4, 60: 5, 61: 12, 62: 1, 63: 3, 67: 6, 68: 1, 70: 4, 71: 1, 72: 19, 73: 22, 74: 4, 75: 3, 76: 3, 78: 2, 80: 1, 82: 11, 83: 11, 84: 2, 85: 2, 86: 6, 88: 5, 89: 2, 90: 3, 91: 4, 92: 2, 93: 14, 94: 3, 95: 14, 96: 3, 98: 5, 99: 2, 100: 6, 101: 3, 102: 9, 103: 55, 104: 64, 105: 16, 106: 5, 107: 38, 108: 25, 109: 18, 110: 26, 111: 76, 112: 57, 113: 40, 114: 21, 115: 149, 116: 46, 117: 9, 118: 6, 119: 19, 120: 37, 121: 32, 122: 15, 123: 92, 124: 72, 165: 10, 166: 4, 167: 6, 168: 7, 169: 7, 171: 2, 172: 3, 173: 10, 174: 10, 175: 3, 177: 1, 178: 20, 180: 6, 181: 2, 183: 2, 184: 4, 185: 7, 186: 8, 187: 3, 189: 8, 191: 5, 192: 1, 194: 8, 195: 4, 196: 2, 197: 9, 198: 4, 199: 4, 200: 4, 201: 1, 206: 10, 208: 3, 209: 18, 210: 2, 211: 4, 212: 9, 213: 8, 214: 2, 215: 3, 216: 1, 217: 8, 220: 3, 221: 11, 222: 2, 224: 13, 225: 44, 226: 63, 227: 57, 228: 4, 229: 6, 230: 21, 231: 32, 233: 52, 234: 105, 235: 58, 236:

,Element #,Number of Patients
0,41,6
1,42,6
2,44,2
3,45,2
4,47,4
...,...,...
731,1149,45
732,1150,3
733,1151,5
734,1152,3


In [2]:
grouping = pd.read_table("public/data/All_patient_data_fre/Readme_element_grouping.txt").iloc[:,0]
grouping

0                                   element_groups_1 = {
1                       #"upper_arm_L": range(721, 737),
2                      # "upper_arm_R": range(871, 887),
3         # "foot_L": chain(range(125, 154), range(15...
4          #"foot_R": chain(range(1, 32), range(33, 1...
                             ...                        
146                                 "Right_2_toe": 1150,
147                                 "Right_3_toe": 1151,
148                                 "Right_4_toe": 1152,
149                            "Right_little_toe": 1153,
150                                                    }
Name: # Group by element number and add specified element numbers to all, Length: 151, dtype: object

In [3]:
for line in grouping:
    if '"' not in line:
        continue
    name = re.search(r'\"(.+?)\"', line).group(1)
    total = 0
    ranges = re.findall(r'range\((\d+), (\d+)\)', line)
    if ranges:
        for range in ranges:
            start, end = map(int, range)
            value = df.loc[df["Element #"].between(start, end, inclusive="left"), "Number of Patients"].sum()
            total += value
    else:
        indexes = re.findall(r'(\d+)', line)
        for index in indexes:
            value = df.loc[df["Element #"] == int(index), "Number of Patients"].sum()
            total += value
    results[name] = int(total)
    results["_".join(reversed(name.split("_")))] = int(total)
with open("element_patient_counts.json", "w") as f:
    json.dump(results, f, indent=4)